In [6]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql import SparkSession

In [13]:
builder = (
    SparkSession
    .builder
    .master("spark://spark:7077")
    .appName("rawApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [14]:
df = spark.read.csv("/opt/workspace/data/input/Iris.csv", header=True)

In [15]:
df.show()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

In [5]:
df.write.mode("overwrite").format("delta").save("/opt/workspace/data/raw/iris/")

In [ ]:
delta_table = DeltaTable.forPath(spark, "/opt/workspace/data/raw/iris/")

In [18]:
delta_table.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      4|2023-05-27 04:16:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          3|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      3|2023-05-27 04:06:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          2|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      2|2

In [19]:
delta_table.optimize().executeCompaction()
delta_table.vacuum()

DataFrame[]

In [ ]:
spark.sql("ALTER TABLE delta.`/opt/workspace/data/raw/iris/` SET TBLPROPERTIES (delta.enableChangeDataFeed=true)")

In [16]:
spark.stop()